In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

In [41]:
# Importing the text the lazy way.
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en')

# Parsing Gatsby.
gatsby = parser(gatsby)

# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words=None,
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [42]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])


(0.07685900359089647, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again.')


In [43]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')
        



done!


In [45]:

# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

[(0.012538179113556777, promises), (0.012538179113556777, exempt), (0.012455008769377494, glimpses), (0.012201713657423653, intimate), (0.012173670024239861, sort)]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


# Drill:

It is also possible that keyword phrases will work better. Modfiy the keyword extraction code to extract two-word phrases (digrams) rather than single words. Then try it with trigrams. You will probably want to broaden the window that defines 'neighbors.' Try a few different modifications, and write up your observations in your notebook. Discuss with your mentor.

In [10]:
import re

def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[token[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [14]:
import re
from nltk.util import ngrams

In [56]:
from nltk import bigrams
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."




In [59]:
bigrm = list(nltk.bigrams(gatsby.split()))

bigrm

[('In', 'my'),
 ('my', 'younger'),
 ('younger', 'and'),
 ('and', 'more'),
 ('more', 'vulnerable'),
 ('vulnerable', 'years'),
 ('years', 'my'),
 ('my', 'father'),
 ('father', 'gave'),
 ('gave', 'me'),
 ('me', 'some'),
 ('some', 'advice'),
 ('advice', 'that'),
 ('that', "I've"),
 ("I've", 'been'),
 ('been', 'turning'),
 ('turning', 'over'),
 ('over', 'in'),
 ('in', 'my'),
 ('my', 'mind'),
 ('mind', 'ever'),
 ('ever', 'since.'),
 ('since.', '"Whenever'),
 ('"Whenever', 'you'),
 ('you', 'feel'),
 ('feel', 'like'),
 ('like', 'criticizing'),
 ('criticizing', 'any'),
 ('any', 'one,"'),
 ('one,"', 'he'),
 ('he', 'told'),
 ('told', 'me,'),
 ('me,', '"just'),
 ('"just', 'remember'),
 ('remember', 'that'),
 ('that', 'all'),
 ('all', 'the'),
 ('the', 'people'),
 ('people', 'in'),
 ('in', 'this'),
 ('this', 'world'),
 ('world', "haven't"),
 ("haven't", 'had'),
 ('had', 'the'),
 ('the', 'advantages'),
 ('advantages', 'that'),
 ('that', "you've"),
 ("you've", 'had."'),
 ('had."', 'He'),
 ('He', "didn

In [60]:
words=set(bigrm)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in bigrm]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [61]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])


[(0.0021008403361344485, ('younger', 'and')), (0.0021008403361344485, ('young', 'men')), (0.0021008403361344485, ("you've", 'had."')), (0.0021008403361344485, ('you', 'feel')), (0.0021008403361344485, ('years', 'my'))]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


This is not more helpful, likely due to the large number of stopwords. "Younger and" "Young men" "You've had" "you feel" and "years my" are not super helpful in summarizing Gatsby.

In [63]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

gatsby = gatsby.lower()
gatsby = re.sub(r'[^a-zA-Z0-9\s]', ' ', gatsby)

stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(gatsby) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
   
 
print(filtered_sentence) 


['younger', 'vulnerable', 'years', 'father', 'gave', 'advice', 'turning', 'mind', 'ever', 'since', 'whenever', 'feel', 'like', 'criticizing', 'one', 'told', 'remember', 'people', 'world', 'advantages', 'say', 'always', 'unusually', 'communicative', 'reserved', 'way', 'understood', 'meant', 'great', 'deal', 'consequence', 'inclined', 'reserve', 'judgments', 'habit', 'opened', 'many', 'curious', 'natures', 'also', 'made', 'victim', 'veteran', 'bores', 'abnormal', 'mind', 'quick', 'detect', 'attach', 'quality', 'appears', 'normal', 'person', 'came', 'college', 'unjustly', 'accused', 'politician', 'privy', 'secret', 'griefs', 'wild', 'unknown', 'men', 'confidences', 'unsought', 'frequently', 'feigned', 'sleep', 'preoccupation', 'hostile', 'levity', 'realized', 'unmistakable', 'sign', 'intimate', 'revelation', 'quivering', 'horizon', 'intimate', 'revelations', 'young', 'men', 'least', 'terms', 'express', 'usually', 'plagiaristic', 'marred', 'obvious', 'suppressions', 'reserving', 'judgments

In [67]:
bigrm_nostp = list(nltk.bigrams(filtered_sentence))

In [69]:
words=set(bigrm_nostp)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(filtered_sentence):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in bigrm_nostp]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [70]:
# Running TextRank
# adjacency.as_matrix will be removed in future version
# use .values instead
nx_words = nx.from_numpy_matrix(adjacency.values())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

[(0.004504504504504488, ('younger', 'vulnerable')), (0.004504504504504488, ('young', 'men')), (0.004504504504504488, ('years', 'father')), (0.004504504504504488, ('world', 'uniform')), (0.004504504504504488, ('world', 'advantages'))]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


#### Now with trigrams

In [73]:
trigrams=list(ngrams(filtered_sentence,3))

In [74]:
trigrams

[('younger', 'vulnerable', 'years'),
 ('vulnerable', 'years', 'father'),
 ('years', 'father', 'gave'),
 ('father', 'gave', 'advice'),
 ('gave', 'advice', 'turning'),
 ('advice', 'turning', 'mind'),
 ('turning', 'mind', 'ever'),
 ('mind', 'ever', 'since'),
 ('ever', 'since', 'whenever'),
 ('since', 'whenever', 'feel'),
 ('whenever', 'feel', 'like'),
 ('feel', 'like', 'criticizing'),
 ('like', 'criticizing', 'one'),
 ('criticizing', 'one', 'told'),
 ('one', 'told', 'remember'),
 ('told', 'remember', 'people'),
 ('remember', 'people', 'world'),
 ('people', 'world', 'advantages'),
 ('world', 'advantages', 'say'),
 ('advantages', 'say', 'always'),
 ('say', 'always', 'unusually'),
 ('always', 'unusually', 'communicative'),
 ('unusually', 'communicative', 'reserved'),
 ('communicative', 'reserved', 'way'),
 ('reserved', 'way', 'understood'),
 ('way', 'understood', 'meant'),
 ('understood', 'meant', 'great'),
 ('meant', 'great', 'deal'),
 ('great', 'deal', 'consequence'),
 ('deal', 'consequenc

In [75]:
words=set(trigrams)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(filtered_sentence):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in trigrams]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [78]:
# Running TextRank
# adjacency.as_matrix will be removed in future version
# use .values instead
nx_words = nx.from_numpy_matrix(adjacency.values)
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

[(0.004524886877828045, ('younger', 'vulnerable', 'years')), (0.004524886877828045, ('young', 'men', 'least')), (0.004524886877828045, ('years', 'father', 'gave')), (0.004524886877828045, ('world', 'uniform', 'sort')), (0.004524886877828045, ('world', 'advantages', 'say'))]


### Neighbors: within 2 places of the target word

In [79]:
words=set(trigrams)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(filtered_sentence):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in trigrams]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+3)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [80]:
# Running TextRank
# adjacency.as_matrix will be removed in future version
# use .values instead
nx_words = nx.from_numpy_matrix(adjacency.values)
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

[(0.004524886877828045, ('younger', 'vulnerable', 'years')), (0.004524886877828045, ('young', 'men', 'least')), (0.004524886877828045, ('years', 'father', 'gave')), (0.004524886877828045, ('world', 'uniform', 'sort')), (0.004524886877828045, ('world', 'advantages', 'say'))]


I don't believe the corpus is large enough for playing with the neighbors to matter